# Описание проекта

## Цель:
- Сопоставление произвольных гео названий с унифицированными именами geonames для внутреннего использования Карьерным центром





## Задачи:


- Создать решение для подбора наиболее подходящих названий с geonames. Например Ереван -> Yerevan


- На примере РФ и стран наиболее популярных для релокации - Беларусь, Армения, Казахстан, Кыргызстан, Турция, Сербия. Города с населением от 15000 человек (с возможностью масштабирования на сервере заказчика)


- Возвращаемые поля geonameid, name, region, country, cosine similarity
- формат данных на выходе: список словарей, например [{dict_1}, {dict_2}, …. {dict_n}] где словарь - одна запись с указанными полями


# imports

In [1]:
import pandas as pd
import numpy as np

import psycopg2 as ps

from GeoSearcher import GeoSearcher

# Демонстрация модели

## Создание подключения

In [2]:
connection = ps.connect(dbname='dbname',
                        user='user',
                        password= 'password',
                        host='localhost',
                        port='5432')

list_of_countrys = ['Russia','Belarus','Armenia',
                    'Kyrgyzstan','Georgia','Kazakhstan',
                    'Turkmenistan','Turkey','Serbia']

## Инициализация модели

In [ ]:
geo = GeoSearcher(connection, list_of_countrys=list_of_countrys,
                  model='LaBSE', translator=False)

In [4]:
res = geo.match_name('Москва')

In [5]:
res

[{'name': array(['Moscow', 'Moskovskiy', 'Mostovskoy', 'Minsk', 'Mozhaysk'],
        dtype=object),
  'region': array(['Moscow', 'Moscow', 'Krasnodar Krai', 'Minsk City',
         'Moscow Oblast'], dtype=object),
  'country': array(['Russia', 'Russia', 'Russia', 'Belarus', 'Russia'], dtype=object),
  'similarity': tensor([0.9729, 0.7980, 0.7014, 0.6948, 0.6617])}]

In [6]:
geo.top_n_recommendations[['asciiname','region','country']]

,asciiname,region,country
20141,Moscow,Moscow,Russia
20169,Moskovskiy,Moscow,Russia
17363,Mostovskoy,Krasnodar Krai,Russia
4439,Minsk,Minsk City,Belarus
19694,Mozhaysk,Moscow Oblast,Russia


# Оценка работы модели

## Загрузка тестовых данных

In [7]:
tests = pd.read_csv('assets\\geo_test.csv', sep=';')

In [8]:
tests.rename(columns={'name':'asciiname'}, inplace=True)

# Оценка

In [9]:
def recall_at_k(predicted, correct,):
        return int(correct in predicted[:5])

In [10]:
total_recall = []
count = 0
for index, row in tests.iterrows():
     geo.match_name(row['query'])
     if geo.top_n_recommendations.loc[(geo.top_n_recommendations['asciiname'] == row['asciiname']) &
                                       (geo.top_n_recommendations['region'] == row['region']) & 
                                       (geo.top_n_recommendations['country'] == row['country'])].shape[0] > 0:
          total_recall.append(1)
     else:
          total_recall.append(0)

In [11]:
np.mean(total_recall)

0.8563218390804598

Мы получили оценку `0.856` для топ-5.

Дообучим модель на альтернативных названиях городов и посмотрим, что получится:

In [12]:
# Создадим пары название города-альтернативное название
#  examples = []
# for index in data[data.alternatenames.notna()].index:
#   for alter_name in data.alternatenames[index].split(','):
#     examples.append([data.loc[index,'asciiname'].lower().strip(), alter_name.lower().strip()])

# Добавляем их тренировчной сет
# train_examples = []
# for ex in examples:
#   train_examples.append(InputExample(texts=ex))

# Определяем функцию ошибки
# train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
# train_loss = losses.MegaBatchMarginLoss(model)

# Дообучаем модель и сохраняем её
# model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=5, warmup_steps=100)
# model.save('/content/model')

In [ ]:
geo = GeoSearcher(connection, list_of_countrys=list_of_countrys,
                  model='model_fine_tune', translator=False)

In [14]:
total_recall = []
count = 0
for index, row in tests.iterrows():
     geo.match_name(row['query'])
     if geo.top_n_recommendations.loc[geo.top_n_recommendations['asciiname'] == row['asciiname']].shape[0] > 0:
          total_recall.append(1)
     else:
          total_recall.append(0)

In [15]:
geo.top_n_recommendations

,name,asciiname,alternatenames,latitude,longitude,fclass,fcode,country_short,cc2,admin1,...,population,elevation,gtopo30,timezone,moddate,iso,country,admin_code,region,asciiname_embeddings
20169,Moskovskiy,Moskovskiy,"Moskovskij,Moskovskiy,Московский",55.59911,37.35495,P,PPL,RU,None,48,...,15435,NaN,185,Europe/Moscow,2015-08-23,RU,Russia,RU.48,Moscow,"[0.017886139452457428, 0.03329120948910713, -0..."
20141,Moscow,Moscow,"MOW,Maeskuy,Maskav,Maskava,Maskva,Mat-xco-va,M...",55.75222,37.61556,P,PPLC,RU,None,48,...,10381222,NaN,144,Europe/Moscow,2022-12-10,RU,Russia,RU.48,Moscow,"[0.03161464259028435, 0.031700558960437775, -0..."
20785,Myski,Myski,"Miski,Moski,Myski,Mõski,Tomazak,mei si ji,mysk...",53.70900,87.80140,P,PPL,RU,None,29,...,44082,NaN,239,Asia/Novokuznetsk,2019-09-05,RU,Russia,RU.29,Kuzbass,"[-0.02779707871377468, 0.04548143967986107, -0..."
16043,Massy,Massy,"Lenin-Dzhol,Leninjol,Massy,Ленин-Джол,Массы",41.06010,72.63285,P,PPLA2,KG,None,03,...,19774,NaN,690,Asia/Bishkek,2022-09-08,KG,Kyrgyzstan,KG.03,Jalal-Abad,"[-0.04320581629872322, 0.05210809409618378, 0...."
4422,Mosty,Mosty,"Mastai,Masti,Masty,Mosti,Mosty,mo si te,Масти,...",53.41220,24.53870,P,PPLA2,BY,None,03,...,15770,NaN,118,Europe/Minsk,2023-02-18,BY,Belarus,BY.03,Grodnenskaya,"[0.017265550792217255, 0.01189790666103363, 0...."


In [16]:
np.mean(total_recall)

0.8850574712643678

После fine_tune модель стала показывать себя лучше. Наша метрика составляет уже `0.885`, что говорит о том, что практически в 88.5% случаев верный вариант есть в топ-5 предложенных. 

In [ ]:
geo = GeoSearcher(connection, list_of_countrys=list_of_countrys,
                  model='model_fine_tune', translator=True)

In [18]:
total_recall = []
count = 0
for index, row in tests.iterrows():
     geo.match_name(row['query'])
     if geo.top_n_recommendations.loc[geo.top_n_recommendations['asciiname'] == row['asciiname']].shape[0] > 0:
          total_recall.append(1)
     else:
          total_recall.append(0)

In [19]:
np.mean(total_recall)

0.7385057471264368

## Заключение

Стоит отметить, что в нашей используется переводчик и spellchecker. Метрика ниже, однако позволяет находить совпадения для более широкого спектра запросов. Если использовать тестовый датасет на другом языке данный подход, вероятно, покажет себя лучше.

Таким образом, создано решение для подбора наиболее подходящих названий с geonames.